In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-pn40vux1
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-pn40vux1


In [38]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
import matplotlib.pyplot as plt

from nltk import word_tokenize


In [20]:
pip install PyMuPDF

In [37]:
import fitz

path_output = '/content/drive/MyDrive/CV_Parsing/data_pdf_txt/all_cvs_renamed/3.pdf'
with open(r'/content/3.txt', 'w', encoding='utf-8') as f:
    with fitz.open(path_output) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
            text = re.sub(' *| *| *|• *| *| *| *|ü *|● *|� *| *| *|▪ *|\s+-', ' ', text)
            text = re.sub('\n\s+', '\n', text)
        print(text)
        print('-------------------------------------------------------------------------------------')
        f.write(text)

 
PERSONAL INFORMATION 
Full name       : Tạ Bá Trung  
Gender           :  Male 
D.O.B            : 15-3-1993 
Birth place     : Thái Thụy- Thái Bình-Việt Nam 
Phone             : 098-595-9011 
Email             : trungtaba@gmail.com 
EDUCATION 
Hanoi University of Science and Technology (HUST) 
Major: Software Technology 
GPA: 3.29 
Grade: Very good 
CERTIFICATION 
System security expert CEHv7Plus 
EXPERIENCE 
FPT SORTWARE | https://www.fpt-software.com 
From: July to now 2018 
Job title: senior blockchain developer 
Main duties: 
Developing blockchain platform 
Developing Dapp in Ethereum and Hyperledger 
VALYOU X MUSIC | http://www.valyouxchange.com 
From: March to April 2018  
Job title: blockchain developer internship 
Main duties:  
Writing ERC721 and ERC165 token 
Working with front-end and mobile team to develop API 
Writing Ethereum client by using web3js and nodejs 
OPENWAY GROUP | http://openwaygroup.com/ 
From: March 2016 to June 2018 
Job title: consultant  
Main duties: 

In [39]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load('/content/drive/MyDrive/CV_Parsing/model_segment_visualize_demo/word2vec_v8.bin', mmap='r')

In [41]:
def decontracted(phrase):
    # specific
    phrase = re.sub(":|-\s+|,|–\s+|-|\||\/", " ", phrase)
    phrase = re.sub(r"â€|™|“|”|;|ü|\xad|\xa0|\u200b|·|∙|�|●|�|§|•|!|▪|©|\?|\]|\[|\)|\(", "", phrase)
    phrase = re.sub(r"[^\w\s]", " ", phrase)
    
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    
    phrase = re.sub(r"\bhcmc\b", "ho chi minh city ", phrase)
    phrase = re.sub(r"\bhcm\b", "ho chi minh ", phrase)
    phrase = re.sub(r"\bbs\b ", "bachelor of ", phrase)
    phrase = re.sub(r"r’s", "r of", phrase)
    phrase = re.sub(r"\’s", " of", phrase)
    
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " of", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\bim\b", "i am ", phrase)
    phrase = re.sub(r"\bi’m\b", "i am ", phrase)
    
    return phrase


def sent2idx(x):
    xidx = []
    for w in word_tokenize(x):
        if w in wv.key_to_index:
            xidx.append(wv.key_to_index[w])
    return np.array(xidx)

In [42]:
from keras.models import load_model

from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

model = load_model('/content/drive/MyDrive/CV_Parsing/model_segment_visualize_demo/bilstm_crf_v4.h5', custom_objects={'CRF': CRF, 
                                                                                                 'crf_loss': crf_loss , 
                                                                                                 'crf_viterbi_accuracy': crf_viterbi_accuracy
                                                                                                 })

In [43]:
# # tag for model version 2
# idx2tag = {0: 'O', 1: 'I-W', 2: 'I-S', 3: 'B-E', 4: 'B-I', 5: 'I-I', 6: 'B-S', 7: 'B-SU', 8: 'I-E', 9: 'B-W', 10: 'I-SU'}
# tag2idx = {'O': 0, 'I-W': 1, 'I-S': 2, 'B-E': 3, 'B-I': 4, 'I-I': 5, 'B-S': 6, 'B-SU': 7, 'I-E': 8, 'B-W': 9, 'I-SU': 10}

# tag for model version 4
tag2idx = {'I-W': 0, 'nan': 1, 'B-SU': 2, 'I-S': 3, 'B-I': 4, 'I_I': 5, 'B-S': 6, 'I-I': 7, 'I-E': 8, 'I-SU': 9, 'B-E': 10, 'B-Su': 11, 'O': 12, 'B-W': 13} 
idx2tag = {0: 'I-W', 1: 'nan', 2: 'B-SU', 3: 'I-S', 4: 'B-I', 5: 'I_I', 6: 'B-S', 7: 'I-I', 8: 'I-E', 9: 'I-SU', 10: 'B-E', 11: 'B-Su', 12: 'O', 13: 'B-W'}

n_tags = len(tag2idx)
# print(tag2idx, '\nn_tag:' ,n_tags)

In [44]:
import re

test_line = []

nfile = '/content/3.txt'
with open(nfile, 'r', encoding = 'utf-8') as f:
    raw = f.readlines()
    for i in raw:
        clean = decontracted(str(i).lower())
        test_line.append(clean.strip())
    
test_data = []
for i in test_line:
    test_data.append(sent2idx(i))

for i, x in enumerate(test_data):
    if len(x) < 64:
        test_data[i] = np.hstack((x, np.zeros(64-len(x))))


#p = model.predict(np.array([test_data[i]]))
#p = np.argmax(p, axis=-1)
# Visualization
print("{:90}||{:5}".format("sentences", "pre tag"))
print(100 * "=")
lst_skill = []
lst_info = []
lst_edu = []
lst_exp = []
lst_obj = []
for i, w in zip(test_data, test_line):
    p = model.predict(np.array([i]))
    p = np.argmax(p)
    print("{:90}: {:5}".format(w, idx2tag[p]))

    if idx2tag[p]=='B-I' or idx2tag[p]=='I-I':
        lst_info.append(w)
    if idx2tag[p]=='B-S' or idx2tag[p]=='I-S':
        lst_skill.append(w)
    if idx2tag[p]=='B-E' or idx2tag[p]=='I-E':
        lst_edu.append(w)
    if idx2tag[p]=='B-W' or idx2tag[p]=='I-W':
        lst_exp.append(w)
    if idx2tag[p]=='B-SU' or idx2tag[p]=='I-SU':
        lst_obj.append(w)

text1 = ', '.join(lst_info)
text2 = ', '.join(lst_skill)
text3 = ', '.join(lst_edu)
text4 = ', '.join(lst_exp)
text5 = ', '.join(lst_obj)

info = re.sub('\s+', ' ', str(text1))
skill = re.sub('\s+', ' ', str(text2))
edu = re.sub('\s+', ' ', str(text3))
exp = re.sub('\s+', ' ', str(text4))
obj = re.sub('\s+', ' ', str(text5))

sentences                                                                                 ||pre tag
                                                                                          : O    
personal information                                                                      : B-I  
full name         tạ bá trung                                                             : I-I  
gender              male                                                                  : I-I  
d o b              15 3 1993                                                              : I-I  
birth place       thái thụy thái bình việt nam                                            : I-I  
phone               098 595 9011                                                          : I-I  
email               trungtaba gmail com                                                   : I-I  
education                                                                                 : B-E  
hanoi university o

In [45]:
print(info)
print(skill)
print(edu)
print(exp)

personal information, full name tạ bá trung, gender male, d o b 15 3 1993, birth place thái thụy thái bình việt nam, phone 098 595 9011, email trungtaba gmail com
system security expert cehv7plus, mysql, key skills, knowledge about blockchain ethereum hyperledger and eos platform, strong knowledge about algorithm, knowledge about object oriented programming distributed application, development, parallel programming openmp mpi cuda, knowledge about design pattern, knowledge about ips card payment system and card business, knowledge about machine learning
education, hanoi university of science and technology hust, major software technology, gpa 3 29, grade very good, program languages java c c python nodejs golang
experience, fpt sortware https www fpt software com, from july to now 2018, job title senior blockchain developer, main duties, developing blockchain platform, developing dapp in ethereum and hyperledger, valyou x music http www valyouxchange com, from march to april 2018, job 

In [46]:
import nltk
nltk.download('stopwords')

def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"', " ", str(input_text))
    
    word_tokens = nltk.tokenize.word_tokenize(word_tokens)
    
    
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # create a set to keep the results in.
    found_skills = set()

    # search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)

    # search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
#Extract Skills
skills = pd.read_csv('/content/drive/MyDrive/CV_Parsing/model_segment_visualize_demo/skills_data (1).csv', encoding='latin')
SKILLS_DB = list(set(skills['Skills'].values))

df_skill = pd.DataFrame(columns=['extract_skill'])
skills1 = extract_skills(skill)
skills2 = extract_skills(exp)
    
# z = str(i) + ''.join(str(j))
join_skills = skills1.union(skills2)
df_skill = df_skill.append({'extract_skill': join_skills}, ignore_index=True)
#     df_skill = df_skill.append({'skill': z, 'extract_skill': join_skills}, ignore_index=True)


#Extract Education
edu_pattern = re.compile('college|master|bachelor|bachelors|.*major.*')
df_edu = pd.DataFrame(columns=['extract_edu'])

lst_edu = []
lst = str(edu).split(',')
for j in lst:
      edu = edu_pattern.findall(j)
      if bool(edu) == True:
          lst_edu.extend(edu)
        
df_edu = df_edu.append({'extract_edu': lst_edu}, ignore_index=True)
#     df_edu = df_edu.append({'education': i, 'extract_edu': lst_edu}, ignore_index=True)


#Extract Experience
year_pattern = re.compile('([2]{1}[0]{1}[0-9]{2})')
exp_pattern = re.compile('([0-9]{2})\s+year')

df_exp = pd.DataFrame(columns=['experience', 'extract_exp'])
lst_exp = []
text = str(edu) + ' ' + str(exp)
exp = exp_pattern.findall(text)
years = year_pattern.findall(text)
    
if bool(exp)==True:
    to_int = [int(i) for i in exp]
    extract_year = exp
    n_exp = max(to_int)
else:
    to_int = [int(i) for i in years]
    extract_year = years
    if len(to_int) > 0:
        max_exp = max(to_int)
        min_exp = min(to_int)
        n_exp = max_exp - min_exp
    else:
        n_exp = 'None'
            
lst_exp.append(str(n_exp))
df_exp = df_exp.append({'extract_exp': lst_exp}, ignore_index=True)


In [48]:
extracted_data = pd.DataFrame({'Extract_Skill': df_skill['extract_skill'],
                               'Extract_Edu': df_edu['extract_edu'],
                              'Extract_Exp': df_exp['extract_exp'],
                              })

In [49]:
pd.set_option("display.max_colwidth", -1)
extracted_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.



,Extract_Skill,Extract_Edu,Extract_Exp
0,"{mysql, writing, blockchain, object oriented programming, mongodb, music, oracle, com, cuda, machine learning, java, hibernate}",[ major software technology],[3]
